In [0]:
CREATE OR REPLACE TABLE infact_dev.marsci.gm_ev_energy_audience AS (
WITH indiv_real_id_mapping AS (
SELECT 
a.GM_PERSON_REALID AS gm_person_real_id
, b.INDIV_BUSINESS_ID AS indiv_id
, a.AP016727_03
FROM (SELECT * FROM dataproducts_dev.bronze_acxiom.gm_consumer_list WHERE snapshot = (
  SELECT MAX(snapshot) 
  FROM dataproducts_dev.bronze_acxiom.gm_consumer_list
  )
  ) a
JOIN (SELECT * FROM dataproducts_dev.bronze_acxiom.gm_owners WHERE snapshot = (
  SELECT MAX(snapshot) 
  FROM dataproducts_dev.bronze_acxiom.gm_owners
  )
  ) b
ON a.GM_PERSON_REALID = b.entity_realID_person_id
)

, gm_ev_owners AS 
(
WITH gm_ev_owners_by_indiv_id AS 
(
SELECT
  individual_business_id as indiv_id,
  umf_xref_vin_book_mktg_div,  -- Vehicle Brand
  umf_xref_vin_book_brand,  -- Vehicle Nameplate
  ev_type_cd
FROM acquire.vehicle_silver.customer_vin_relation acq
JOIN reporting.vehicle360.fs_veh_ident veh
ON acq.vin = veh.veh_id_nbr
WHERE 1 = 1
AND disposal_dt IS NOT NULL
AND ev_type_cd IS NOT NULL  -- NULL indicates ICE 
)

SELECT b.gm_person_real_id
, b.AP016727_03
FROM gm_ev_owners_by_indiv_id a
JOIN indiv_real_id_mapping b
ON a.indiv_id = b.indiv_id
)


, gm_ev_intenders AS 
(
WITH gm_ev_intenders_by_real_id AS 
(
SELECT gm_person_real_id
FROM infact_dev.marsci.pronghorn_audiences_history
WHERE Data_YrMonth = (
  SELECT MAX(Data_YrMonth) 
  FROM infact_dev.marsci.pronghorn_audiences_history
  )
AND audience_name IN ('cb_gm_energy_1p', 'cb_gm_energy_3p')
)

SELECT a.gm_person_real_id
, b.AP016727_03
FROM gm_ev_intenders_by_real_id a
JOIN (SELECT * FROM dataproducts_dev.bronze_acxiom.gm_consumer_list WHERE snapshot = (
  SELECT MAX(snapshot) 
  FROM dataproducts_dev.bronze_acxiom.gm_consumer_list
  )
  ) b
ON a.gm_person_real_id = b.GM_PERSON_REALID
)


, gm_bidirectional_audience AS 
(
SELECT * FROM gm_ev_intenders
UNION DISTINCT
(
SELECT * FROM gm_ev_owners
)
)

SELECT gm_person_real_id FROM gm_bidirectional_audience
WHERE AP016727_03 = 10
)

-- AP016727_13

-- DECILE 1 (Highest probabilities to be energy efficient): 4.1M
-- DECILE 2 (2nd Highest probabilities to be energy efficient): 2.8M
-- DECILE 3 (3rd Highest probabilities to be energy efficient): 1.6M
-- DECILE 4 (4th Highest probabilities to be energy efficient): 0.8M
-- DECILE 5 (5th Highest probabilities to be energy efficient): 0.4M
-- DECILE 6 (6th Highest probabilities to be energy efficient): 0.2M
-- DECILE 7 (7th Highest probabilities to be energy efficient): 0.1M
-- DECILE 8 (8th Highest probabilities to be energy efficient): 50K
-- DECILE 9 (9th Highest probabilities to be energy efficient): 20K
-- DECILE 10 (10th Highest probabilities to be energy efficient): 7K

-- RZP4VJ, Brad